# Create a Service to transcrbe the earnings call
The whisper service has already transcribed sound files relating to the earnings calls.  It was hosted on a snowpark container and it processed each file with a medium sized GPU.  Whisper leverages pytorch (a deap leaning framework) to parse and transcribe the text.

***COMING SOON*** - Soon, a new multi modal **Transcribe** cortex function will be available.  This will mean that one simple function will allow you to transcribe sound from a file **WITHOUT ANY SETUP**.

For today, we will use the transcripts which were processed in advance of this lab.  For reference, the results are shared to you via the **Internal Marketplace**

***FOR INFORMATION***

**Snowflake Data Foundations** is a product on the marketplace which include event transcripts.  There is a 30 day trial available.

Click [here](https://app.snowflake.com/marketplace/listing/GZTSZ290BUX66/snowflake-data-snowflake-data-foundations?search=foundations) to find out more.

####  Listen to the calls that were transcribed

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


files = session.sql('''SELECT RELATIVE_PATH, GET_PRESIGNED_URL('@DOCUMENT_AI.EARNINGS_CALLS',RELATIVE_PATH) URL FROM DIRECTORY (@DOCUMENT_AI.EARNINGS_CALLS)''')


select_call = st.selectbox('Select Call:', files.select('RELATIVE_PATH'))

URL = files.filter(col('RELATIVE_PATH') == select_call).select('URL').collect()[0][0]
st.audio(URL, format="audio/mpeg")

#### The sound files have already been transcribed using the whisper service.
We will now create a table based on the raw results produced by the Whisper Service. 

In [ ]:
SELECT * FROM DEFAULT_SCHEMA.EARNINGS_CALL_TRANSCRIPT

### Transform the transcript table
We are creating a table which is flattening the generated whisper data.  This creates a row for every snippet of commentary.

In [ ]:
CREATE TABLE IF NOT EXISTS DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS AS

SELECT RELATIVE_PATH, PARSE_JSON(TRANSCRIPT):language::TEXT LANGUAGE,

VALUE:end::FLOAT TIME_SECONDS,  
VALUE:text::TEXT TEXT 
FROM DEFAULT_SCHEMA.EARNINGS_CALL_TRANSCRIPT,
LATERAL FLATTEN (PARSE_JSON(TRANSCRIPT):segments);

SELECT * FROM DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS LIMIT 5

#### Add Sentiment scores to the calls
Here, Cortex Sentiment is being used to generate a sentiment score for each text snippet

In [ ]:
SELECT *, SNOWFLAKE.CORTEX.SENTIMENT(TEXT) FROM DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS

#### Put all together in Streamlit
Let's now have a look at the results using a visualistion in Streamlit

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

def sentiment(text):
    return call_function('snowflake.cortex.sentiment',text)

transcript_with_sentiment = session.table('DEFAULT_SCHEMA.TRANSCRIBED_TRANSCRIPTS').with_column('sentiment',sentiment(col('TEXT')))

st.markdown('#### Calls with Sentiment')


st.dataframe(transcript_with_sentiment)
col1,col2,col3 = st.columns(3)

with col1:

    st.markdown('#### Q1')
    q1 = transcript_with_sentiment.filter(col('RELATIVE_PATH')=='EARNINGS_Q1_FY2025.mp3')
    st.line_chart(q1,
              y='SENTIMENT',x='TIME_SECONDS',color = '#29B5E8')
    st.metric('Average Sentiment',q1.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col2:
    q2 = transcript_with_sentiment.filter(col('RELATIVE_PATH')=='EARNINGS_Q2_FY2025.mp3')
    st.markdown('#### Q2')
    st.line_chart(q2,
              y='SENTIMENT',x='TIME_SECONDS',color = '#29B5E8')
    st.metric('Average Sentiment',q2.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col3:
    
    st.markdown('#### Q3')
    q3 = transcript_with_sentiment.filter(col('RELATIVE_PATH')=='EARNINGS_Q3_FY2025.mp3')
    st.line_chart(q3,
              y='SENTIMENT',x='TIME_SECONDS',color = '#FF9F36')
    st.metric('Average Sentiment',q3.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

You will see that the line charts are very hard to read.  Also with such a small snippet of information, it is difficult to get a good sentiment score.  Unlike the chunking which you did with the Analyst reports, this time we are going to group snippets of data together for every 60 seconds and then create an average sentiment for each minute. 

In [ ]:
grouped = transcript_with_sentiment.with_column('TIME',time_from_parts(15,0,'TIME_SECONDS')).\
with_column('MINUTES',date_trunc('minute','TIME'))
grouped = grouped.with_column('MINUTES',minute('MINUTES'))
data_grouped_minutes = grouped.group_by('RELATIVE_PATH','MINUTES').agg(array_agg('TEXT').alias('TEXT'),avg('SENTIMENT').alias('SENTIMENT'))

st.markdown('''#### Data Grouped to Minutes''')
data_grouped_minutes


Below is a much smoother output - and a lot easier to read.  We are also able to view these grouped snippets in the visualistion.  Here, we are featuring the Most popular minute of the year, followed by the most negative minute of the year.

In [ ]:
st.markdown('#### Sentiment Analysis during the duration of the last 3 quarterly earnings calls')
col1,col2,col3 = st.columns(3)

with col1:

    st.markdown('#### Q1')
    q1 = data_grouped_minutes.filter(col('RELATIVE_PATH')=='EARNINGS_Q1_FY2025.mp3')
    st.line_chart(q1,
              y='SENTIMENT',x='MINUTES',color = '#29B5E8')
    st.metric('Average Sentiment',q1.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col2:
    q2 = data_grouped_minutes.filter(col('RELATIVE_PATH')=='EARNINGS_Q2_FY2025.mp3')
    st.markdown('#### Q2')
    st.line_chart(q2,
              y='SENTIMENT',x='MINUTES',color = '#29B5E8')
    st.metric('Average Sentiment',q2.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

with col3:
    
    st.markdown('#### Q3')
    q3 = data_grouped_minutes.filter(col('RELATIVE_PATH')=='EARNINGS_Q3_FY2025.mp3')
    st.line_chart(q3,
              y='SENTIMENT',x='MINUTES',color = '#FF9F36')
    st.metric('Average Sentiment',q3.agg(avg('SENTIMENT').alias('SENTIMENT')).select(round('SENTIMENT',2)).collect()[0][0])

st.markdown(f'''**:bulb: Most positive minute of the year**: \
{data_grouped_minutes.sort(col('SENTIMENT').desc()).limit(1).select(array_to_string(col('TEXT'),lit(''))).collect()[0][0]}''')

st.markdown(f'''**:warning: Most negative minute of the year**: \
{data_grouped_minutes.sort(col('SENTIMENT').asc()).limit(1).select(array_to_string(col('TEXT'),lit(''))).collect()[0][0]}''')

Now lets remove the arrays to visualise pure text.

In [ ]:
grouped_text = data_grouped_minutes.with_column(
    'TEXT',
    regexp_replace(cast(col('TEXT'), StringType()), r'[\[\]"]', '')).sort(col('RELATIVE_PATH'),col('MINUTES')
)
grouped_text

We will now put all the minutes of the call together so we can see a complete picture of each call. We also summarise the call and put a final sentiment score for the entire call.  Have a look at the sentiment call when the entire call is put in context.

In [ ]:
entire_call = grouped_text.group_by('RELATIVE_PATH').agg(array_agg('TEXT').alias('TEXT')).with_column('TEXT',array_to_string('TEXT',lit(' ')))

entire_call = entire_call.with_column('SUMMARY',snowflake_cortex_summarize(col('TEXT')))
entire_call = entire_call.with_column('SENTIMENT',call_function('SNOWFLAKE.CORTEX.SENTIMENT',(col('TEXT'))))
entire_call

#### Save data in a table
Finally lets save the results in a table.   Let's examine the text further with text based search functions.

In [ ]:
grouped_text.write.mode("overwrite").save_as_table("DEFAULT_SCHEMA.transcripts_by_minute")
entire_call.write.mode("overwrite").save_as_table("DEFAULT_SCHEMA.full_transcripts")

### VECTOR EMBEDDINGS
The calls will be chunked and then embedded to make them searchable.

An embedding refers to the reduction of high-dimensional data, such as unstructured text, to a representation with fewer dimensions, such as a vector. Modern deep learning techniques can create vector embeddings, which are structured numerical representations, from unstructured data such as text and images, preserving semantic notions of similarity and dissimilarity in the geometry of the vectors they produce.

The illustration below is a simplified example of the vector embedding and geometric similarity of natural language text. In practice, neural networks produce embedding vectors with hundreds or even thousands of dimensions, not two as shown here, but the concept is the same. Semantically similar text yields vectors that “point” in the same general direction.

![embeddings](https://docs.snowflake.com/en/_images/vector-similarity-vectors-example.png)

Below uses cortex **split text recursive character** to split as before, and in addition, uses **embed text 1024** to embed the text to 1024 dimensions.

As the data is chunked based on number of characters with the addition to an overlap, a new sentiment score is also added to the new chunked table.

In [ ]:
entire_call = session.table('DEFAULT_SCHEMA.full_transcripts')
#### chunk the calls ####
chunked = entire_call.with_column('TEXT',call_function('SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER',col('TEXT'),'none',500,20))
chunked = chunked.join_table_function('flatten','TEXT').select('RELATIVE_PATH','SUMMARY',col('VALUE').astype(StringType()).alias('TEXT'))

#### apply sentiment per chunk

chunked = chunked.with_column('SENTIMENT',call_function('SNOWFLAKE.CORTEX.SENTIMENT',col('TEXT')))
#### embed the calls ###

chunked = chunked.with_column('EMBED',call_function('SNOWFLAKE.CORTEX.EMBED_TEXT_1024',lit('snowflake-arctic-embed-l-v2.0'),col('TEXT')))


chunked.write.mode("overwrite").save_as_table("DEFAULT_SCHEMA.call_embeds")
chunked = session.table('DEFAULT_SCHEMA.call_embeds')
chunked.limit(1)

Now you will try and search the call text using the the search box below.  It will compare the embedded search **phrase** with the embeddings of the chunked data.

In [ ]:
call = st.selectbox('Select Call:',entire_call.select('RELATIVE_PATH').distinct())

with st.container(height=900):
    st.markdown('### SUMMARY')
    chunked.filter(col('RELATIVE_PATH')==call).select('SUMMARY').collect()[0][0]
    keyword = st.text_input('Search Text:')
    search = chunked.filter(col('RELATIVE_PATH')==call)
    
    search = search.with_column('embed_search',call_function('SNOWFLAKE.CORTEX.EMBED_TEXT_1024',
                                                     lit('snowflake-arctic-embed-l-v2.0'),
                                                     lit(keyword)).alias('TOKEN'))

    search = search.with_column('score',call_function('VECTOR_COSINE_SIMILARITY',col('EMBED'),col('EMBED_SEARCH')))
    st.markdown('### SEARCH RESULTS')
    st.table(search.select('TEXT','SCORE','SENTIMENT').sort(col('score').desc()).limit(3))

This is the principle of how the search service works.  Cortex search does all the embeddings for the user, this is what you will do in the next section - **Create a Search Service**